# **人工智能实验报告 第五周**


## **一、实验题目**
使用遗传算法（GA）求解旅行商问题，从网站上选取不同.tsp文件进行求解


## **二、实验内容**

本实验实现了遗传算法求解TSP问题，并通过模块化的方式构建了算法的主流程，包括种群初始化、适应度评估、选择、交叉、变异、精英保留、2-opt局部搜索等关键步骤。同时在普通遗传算法的基础上进行了大量的优化，以及不同种子的采用选择统计，针对不同的城市规模，得到一套相应的可行运行策略。

为了更好的实现该问题以及进行数据可视化的展示，本项目在主文件夹下分设了几个不同的项目。`GA`文件为主项目，`data`文件夹包含了测试的`.tsp`文件，`result`文件夹包含了对于每个测试样例导出的种子参数设置，最优路径及其长度，`image`文件夹包含了每个测试样例对应的路径图以及迭代图可视化展示，`code`文件夹为核心代码，分别为`utils.py`辅助函数文件，`selections.py`策略选择文件，`crossover.py`交叉遗传文件，`init_population.py`初始化种群文件，`mutate.py`变异文件，`run_ga.py`整合的函数文件，`test.py`测试文件。

## **三、算法原理与关键代码**

### **3.1 种群初始化（`init_population.py`）**

初始化是遗传算法的第一步，本实验使用了三种方式进行初始个体生成：

- **随机初始化**：直接随机打乱城市编号生成路径
- **最近邻初始化**：从任意一个城市出发，每次选择距离当前城市最近但尚未访问的城市，直到构造出完整路径
- **混合型**：即使用最邻近也使用邻近最远，其余为随机，优化初始种群的数量

这种方式既保证了解的多样性，也提升了解的初始质量，有助于算法快速收敛。


In [1]:
# init_population.py
# 初始化种群文件

import random
import numpy as np

def nearest_neighbor(dist_matrix):
    """
    加入邻近最优
    """
    n = len(dist_matrix)
    unvisited = set(range(n))
    current = random.choice(list(unvisited))
    path = [current]
    unvisited.remove(current)
    while unvisited:
        next = min(unvisited, key=lambda city: dist_matrix[current][city])
        path.append(next)
        unvisited.remove(next)
        current = next
    return path


def farthest_insertion(dist_matrix):
    """
    邻近最远启发式
    """
    n = len(dist_matrix)
    remaining = list(range(n))
    tour = [random.choice(remaining)]
    remaining.remove(tour[0])

    farthest = max(remaining, key=lambda x: dist_matrix[tour[0]][x])
    tour.append(farthest)
    remaining.remove(farthest)

    while remaining:
        next_city = max(remaining, key=lambda x: min(dist_matrix[x][c] for c in tour))

        best_pos, best_cost = 0, float('inf')
        for i in range(len(tour)):
            a, b = tour[i], tour[(i+1) % len(tour)]
            cost = dist_matrix[a][next_city] + dist_matrix[next_city][b] - dist_matrix[a][b]
            if cost < best_cost:
                best_pos, best_cost = i+1, cost

        tour.insert(best_pos, next_city)
        remaining.remove(next_city)
    return tour


def random_shuffle(num_cities):
    """
    随机解
    """
    path = list(range(num_cities))
    random.shuffle(path)
    return path

def init_populations(pop_size, num_size, dist_matrix, strategy):
    """
    初始化种群
    """
    populations = []
    base = np.arange(num_size)
    if strategy == 'nearest':
        populations.append(nearest_neighbor(dist_matrix))
        rand_part = [np.random.permutation(base).tolist() for i in range(pop_size - 1)]
        populations.extend(rand_part)
        return populations
    elif strategy == 'random':
        populations = [np.random.permutation(base).tolist() for _ in range(pop_size)]
        return populations
    elif strategy == 'farthest':
        populations.append(farthest_insertion(dist_matrix))
        rand_part = [np.random.permutation(base).tolist() for i in range(pop_size - 1)]
        populations.extend(rand_part)
        return populations
    else:
        populations.append(farthest_insertion(dist_matrix))
        populations.append(nearest_neighbor(dist_matrix))
        rand_part = [np.random.permutation(base).tolist() for i in range(pop_size - 2)]
        populations.extend(rand_part)
        return populations



### **3.2 距离计算与适应度函数（`utils.py`）**

适应度函数用于评估路径的好坏，定义为路径长度的倒数：

$$ \text{fitness} = \frac{1}{\text{total\_distance}} $$

同时路径长度由城市间的欧几里得距离组成，先通过坐标生成一个对称的距离矩阵。


In [2]:
import numpy as np

def distance_cites(cities):
    cities = np.array(cities)
    n = len(cities)
    diff = cities[:, np.newaxis, :] - cities[np.newaxis, :, :]
    dist_matrix = np.sqrt(np.sum(diff ** 2, axis=2))
    return dist_matrix

def get_total(dist_matrix, path):
    path = np.array(path)
    next_path = np.roll(path, -1)
    return np.sum(dist_matrix[path, next_path])

def fitness(dist_matrix, path):
    return 1.0 / get_total(dist_matrix, path)


def population_diversity(population):
    """
    多样性测试
    """
    total_diff = 0
    n = len(population[0])
    base = population[0]
    for ind in population[1:]:
        diff = sum(1 for i, j in zip(base, ind) if i != j)
        total_diff += diff
    return total_diff / ((len(population) - 1) * n)


def load_tsp_file(filepath):
    cities = []
    with open(filepath, 'r') as file:
        lines = file.readlines()
        start = False
        for line in lines:
            if line.strip() == "NODE_COORD_SECTION":
                start = True
                continue
            if line.strip() == "EOF":
                break
            if start:
                parts = line.strip().split()
                if len(parts) >= 3:
                    x = float(parts[1])
                    y = float(parts[2])
                    cities.append((x, y))
    return cities

def plot_tsp_path(cities, path, title="Best Path", save_path=None):
    x = [cities[i][0] for i in path + [path[0]]]
    y = [cities[i][1] for i in path + [path[0]]]
    plt.figure(figsize=(8, 6))
    plt.plot(x, y, marker='o', linestyle='-', linewidth=1.5)
    plt.title(title)
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.grid(True)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
    else:
        plt.show()
    plt.close()

def plot_convergence(best_list, avg_list, save_path=None):
    plt.figure(figsize=(8, 5))
    plt.plot(best_list, label="Best Path Length")
    plt.plot(avg_list, label="Average Path Length")
    plt.title("GA Convergence")
    plt.xlabel("Generation")
    plt.ylabel("Distance")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
    else:
        plt.show()
    plt.close()


### **3.3 选择策略（`selections.py`）**

本实验采用**锦标赛选择**（Tournament Selection）：

1. 每次从种群中随机选出若干个个体（如5个）
2. 从中选出适应度最高者进入下一代

该策略在保留优秀个体的同时控制了过度集中的风险。


In [3]:
def selection(populations, dist_matrix):
    """
    轮盘赌
    """
    fits = [fitness(dist_matrix, path) for path in populations]
    total = sum(fits)
    new_populatiions = []
    for i in range(len(populations)):
        pick = random.random() * total
        current = 0
        for path, f in zip(populations, fits):
            current += f
            if current > pick:
                new_populatiions.append(path[:])
                break
    return new_populatiions

def tournament_selcetion(populations, dist_matrix, size=3):
    """
    锦标赛
    """
    new_population = []
    n = len(populations)
    for i in range(n):
        tournament = random.sample(populations, size)
        winner = max(tournament, key=lambda ind: fitness(dist_matrix, ind))
        new_population.append(winner[:])
    return new_population

### **3.4 交叉操作（`crossover.py`）**

交叉操作用于组合两个父代个体的信息，生成新的解。

本实验支持两种常用方式：
- **PMX（部分映射交叉）**
- **OX（顺序交叉）**

两者均保证产生的子代路径合法，即所有城市无重复且不遗漏。


In [4]:
import random

def pmx_crossover(parent1, parent2):
    """
    映射交叉
    """
    n = len(parent1)
    child1 = [None] * n
    child2 = [None] * n
    start, end = sorted([random.randint(0, n-1) for i in range(2)])
    child1[start:end+1] = parent1[start:end+1]
    child2[start:end+1] = parent2[start:end+1]

    
    def pmx_fill(child, parent_other):
        for i in range(start, end+1):
            tmp = parent_other[i]
            if tmp not in child:
                pos = i
                while True:
                    gen_in_child = child[pos]
                    pos = parent_other.index(gen_in_child)
                    if child[pos] is None:
                        child[pos] = tmp
                        break
        for i in range(n):
            if child[i] is None:
                child[i] = parent_other[i]
        return child
    
    child1 = pmx_fill(child1, parent2)
    child2 = pmx_fill(child2, parent1)
    return child1, child2


def order_crossover(parent1, parent2):
    """
    顺序交叉
    """
    n = len(parent1)
    child1 = [0] * n
    child2 = [0] * n
    start, end = sorted([random.randint(0, n - 1) for _ in range(2)])
    child1[start:end+1] = parent1[start:end+1]
    child2[start:end+1] = parent2[start:end+1]

    pos1 = (end + 1) % n
    for i in parent1:
        if i not in child2:
            child2[pos1] = i
            pos1 = (pos1 + 1) % n
    pos2 = (end + 1) % n
    for i in parent2:
        if i not in child1:
            child1[pos2] = i
            pos2 = (pos2 + 1) % n
    return child1, child2

### **3.5 变异操作（`mutate.py`）**

变异操作引入随机扰动，增加种群多样性，避免陷入局部最优。

本实验实现了：
- **交换变异**：随机交换两个城市位置
- **反转变异**：选定一段区间，反转路径顺序

支持动态调整变异概率以适应不同进化阶段的需要。


In [5]:
def mutate_swap(individual, mutation_rate=0.005) -> None:
    """
    交换变异
    """
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(individual) - 1)
            individual[i], individual[j] = individual[j], individual[i]

def two_opt(route, dist_matrix):
    best = route
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 1, len(route)):
                if j - i == 1:
                    continue
                new_route = route[:]
                new_route[i:j] = route[j-1:i-1:-1]
                if get_total(dist_matrix, new_route) < get_total(dist_matrix, best):
                    best = new_route
                    improved = True
        route = best
    return best


def two_opt_limited(route, dist_matrix, max_swaps=2):
    best = route[:]
    best_dist = get_total(dist_matrix, best)
    count = 0
    n = len(route)

    for i in range(1, n - 2):
        for j in range(i + 1, n):
            if j - i == 1:
                continue
            new_route = best[:]
            new_route[i:j] = reversed(new_route[i:j])
            new_dist = get_total(dist_matrix, new_route)
            if new_dist < best_dist:
                best = new_route
                best_dist = new_dist
                count += 1
                if count >= max_swaps:
                    return best
    return best

def disturb_path(path, strength=0.2):
    path = path[:]
    n = len(path)
    for _ in range(int(strength * n)):
        i, j = sorted(random.sample(range(n), 2))
        path[i:j] = reversed(path[i:j])
    return path

### **3.6 主运行流程（`run_ga.py`）**

主函数 `run_ga` 负责控制整个遗传算法的流程，包括：

- 初始化种群
- 多代进化（选择、交叉、变异、局部搜索）
- 记录最优解与收敛曲线

此外还集成了2-opt优化算法以提升解的局部质量。


In [6]:
def run_ga(dist_matrix, 
           pop_size=300, 
           generations=2000, 
           crossover_rate=0.9, 
           mutation_rate=0.05, 
           elitism_count=3,
           gen_two_opt=200,
           use_two_opt=False,
           two_opt_strategy='Normal',
           init_strategy='random',
           selection_strategy='tournament',
           crossover_strategy='order',
           early_stop=500):
    num_cities = len(dist_matrix)

    best_distance_gen = []
    avg_distance_gen = []

    if init_strategy == 'random':
        population = init_populations(pop_size, num_cities, dist_matrix, 'random')
    elif init_strategy == 'nearest':
        population = init_populations(pop_size, num_cities, dist_matrix, 'nearest')
    elif init_strategy == 'mixed':
        population = init_populations(pop_size, num_cities, dist_matrix, 'mixed')

    best_individual = None
    best_fit = float('-inf')
    last_improven_gen = 0
    original_mutation = mutation_rate

    for gen in range(generations):

        if selection_strategy == 'tournament':
            population = tournament_selcetion(population, dist_matrix)
        else:
            population = selection(population, dist_matrix)

        sorted_pop = sorted(population, key=lambda ind: fitness(dist_matrix, ind), reverse=True)
        elties = [ind[:] for ind in sorted_pop[:elitism_count]] if elitism_count > 0 else []

        current_best = sorted_pop[0]
        current_best_fit = fitness(dist_matrix, current_best)
        if current_best_fit > best_fit:
            best_individual = current_best[:]
            best_fit = current_best_fit
            last_improven_gen = gen
            mutation_rate = original_mutation
        elif gen - last_improven_gen > early_stop // 4:
            disturbed = disturb_path(best_individual, strength=0.3)
            repaired = two_opt_limited(disturbed, dist_matrix, max_swaps=3)
            if fitness(dist_matrix, repaired) > fitness(dist_matrix, best_individual):
                best_individual = repaired
                last_improven_gen = gen
        elif gen - last_improven_gen > early_stop // 2:
            mutation_rate = original_mutation * 3

        diversity = population_diversity(population)
        if diversity < 0.1:
            mutation_rate = original_mutation * 3
            replace_count = max(5, int(0.05 * len(population)))
            for i in range(replace_count):
                population[-(i+1)] = random.sample(range(num_cities), num_cities)
        else:
            mutation_rate = original_mutation



        next_population = elties[:]

        while len(next_population) < pop_size:
            parent1 = random.choice(population)
            parent2 = random.choice(population)
            if random.random() < crossover_rate:
                if crossover_strategy == 'order':
                    child1, child2 = order_crossover(parent1, parent2)
                elif crossover_strategy == 'pmx':
                    child1, child2 = pmx_crossover(parent1, parent2)
                elif crossover_strategy == 'mixed':
                    if random.random() < 0.5:
                        child1, child2 = order_crossover(parent1, parent2)
                    else:
                        child1, child2 = pmx_crossover(parent1, parent2)
                else:
                    child1, child2 = parent1[:], parent2[:]
            else:
                child1, child2 = parent1[:], parent2[:]

            mutate_swap(child1, mutation_rate)
            mutate_swap(child2, mutation_rate)
            next_population.append(child1)
            if len(next_population) < pop_size:
                next_population.append(child2)

        if gen % gen_two_opt == 0 and use_two_opt:
            if two_opt_strategy == 'normal':
                top_k = int(0.01 * len(next_population))
                for i in range(top_k):
                    next_population[i] = two_opt(next_population[i], dist_matrix)
            elif two_opt_strategy == 'limited':
                best_individual = two_opt_limited(best_individual, dist_matrix, max_swaps=2)

        population = next_population

        
        avg_fit = sum(fitness(dist_matrix, ind) for ind in population) / len(population)
        avg_dist = 1.0 / avg_fit

        best_distance_gen.append(1.0 / best_fit)
        avg_distance_gen.append(avg_dist)

        if gen % 100 == 0:
            fits = [fitness(dist_matrix, ind) for ind in population]
            avg_fit = sum(fits) / len(fits)
            print(f"第{gen}代：最佳长度 {1.0 / best_fit:.2f}，平均长度 {1.0 / avg_fit:.2f}")
        

        if gen - last_improven_gen >= early_stop:
            print("提前终止!")
            break
    return best_individual, 1.0 / best_fit, best_distance_gen, avg_distance_gen


def run_ga_on_cities(cities, 
                     pop_size=300,  
                     generations=2000, 
                     crossover_rate=0.9, 
                     mutation_rate=0.05, 
                     elitism_count=3,
                     gen_two_opt=200,
                     use_two_opt=False,
                     two_opt_strategy='limited',
                     init_strategy='random',
                     selection_strategy='tournament',
                     crossover_strategy='order',
                     early_stop=500):
    dist_matrix = distance_cites(cities)
    return run_ga(dist_matrix, pop_size, generations, crossover_rate, mutation_rate, elitism_count, gen_two_opt, use_two_opt, two_opt_strategy, init_strategy, selection_strategy, crossover_strategy,early_stop)


### **3.7 测试运行入口（`test.py`）**

文件`test.py`保存了测试需要的函数，同时增加可视化的操作，输出数据表格以及图像。



## **三、实验结果分析**

该实验采用单独的测试文件`test.py`进行测试，需要将数据文件.tsp下载至文件夹`data`中运行。结果会在终端中显示，同时也将保存为.csv文件以及可视化图像分别保存在`result`与`iamge`文件夹下。

### **3.1运行说明**

本实验使用`test.py`文件进行运行，可调节输入的数据以及参数类型。可以对同一城市的TSP问题进行多次遍历寻找最优结果并输出。输出的结果同时会以.csv文件保存，并且输出相应的路径图与收敛图。

可以调节的设置，包括参数，数据，城市情况等如下。
```python
    cities = load_tsp_file("data/qa194.tsp")
    num_runs = 1
    
    config = {
        'pop_size': 300,
        'generations': 2000,
        'crossover_rate': 0.9,
        'mutation_rate': 0.02,
        'elitism_count': 1,
        'use_two_opt': True,
        'gen_two_opt': 400,
        'two_opt_strategy': 'normal',
        'init_strategy': 'random',
        'selection_strategy': 'tournament',
        'crossover_strategy': 'mixed',
        'early_stop': 600
    }
```
下面提供一个可以运行的结果代码（包括输出可视化的.csv文件以及图片，同时代码中增加了每百代输出路径信息）：

In [7]:
import csv

# 导入代码
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..', 'code')) 

from run_ga import run_ga_on_cities
from utils import load_tsp_file, plot_convergence, plot_tsp_path


if __name__ == '__main__':
    cities = load_tsp_file(os.path.join(os.getcwd(), '..', 'data', 'dj38.tsp')) # 可以调节
    num_runs = 1
    results = []

    
    config = {
        'pop_size': 300,
        'generations': 2000,
        'crossover_rate': 0.9,
        'mutation_rate': 0.02,
        'elitism_count': 0,
        'use_two_opt': False,
        'gen_two_opt': 400,
        'two_opt_strategy': 'normal',
        'init_strategy': 'random',
        'selection_strategy': 'tournament',
        'crossover_strategy': 'mixed',
        'early_stop': 600
    }


    os.makedirs(os.path.join(os.getcwd(), '..', 'result'), exist_ok=True)
    os.makedirs(os.path.join(os.getcwd(), '..', 'image'), exist_ok=True)

    
    csv_filename = os.path.join(os.getcwd(), '..', 'result', f"results_{len(cities)}cities_{num_runs}runs_testfornotebook.csv")

    with open(csv_filename, "w", newline='', encoding='utf-8') as f:
        writer = csv.writer(f)

        writer.writerow(["Parameter Settings"])
        for key, value in config.items():
            writer.writerow([key, value])
        
        writer.writerow([])

        writer.writerow(["Run", "Best_Distance", "Best_Path"])

        for i in range(num_runs):
            best_path, best_dist, best_curve, avg_curve = run_ga_on_cities(cities, **config)
            results.append((best_dist, best_path))
            writer.writerow([i + 1, best_dist, best_path])
            print(f"[Run {i+1}] Distance: {best_dist:.2f}")

            city_count = len(cities)
            path_img = os.path.join(os.getcwd(), '..', 'image', f"path_{city_count}cities_run{i+1}_2.png")
            curve_img = os.path.join(os.getcwd(), '..', 'image', f"convergence_{city_count}cities_run{i+1}_2.png")

            plot_tsp_path(cities, best_path, title=f"Best Path (Run {i+1})", save_path=path_img)
            plot_convergence(best_curve, avg_curve, save_path=curve_img)

        results.sort(key=lambda x: x[0])
        best_overall = results[0]
        worst_overall = results[-1]
        avg_distance = sum(r[0] for r in results) / len(results)

        writer.writerow([])
        writer.writerow(["Summary"])
        writer.writerow(["Best_Distance", best_overall[0]])
        writer.writerow(["Worst_Distance", worst_overall[0]])
        writer.writerow(["Average_Distance", avg_distance])
        writer.writerow(["Best_Path", best_overall[1]])

    print(f"Best : {best_overall[0]:.2f}")
    print(f"Worst: {worst_overall[0]:.2f}")
    print(f"Avg  : {avg_distance:.2f}")


第0代：最佳长度 21660.76，平均长度 26487.81
第100代：最佳长度 16130.91，平均长度 21853.13
第200代：最佳长度 14671.45，平均长度 23119.62
第300代：最佳长度 14671.45，平均长度 22234.91
第400代：最佳长度 14671.45，平均长度 22789.96
第500代：最佳长度 14671.45，平均长度 22647.02
第600代：最佳长度 14671.45，平均长度 21803.44
第700代：最佳长度 14671.45，平均长度 22422.70
提前终止!
[Run 1] Distance: 14671.45
Best : 14671.45
Worst: 14671.45
Avg  : 14671.45


### **3.2结果分析**

### **3.21大致分析**
下面给出不同规模的城市测试数据，对于规模在$1000$左右的数据，算法都能得到一个很接近正确的解，但是当城市规模增大至$10000$左右，结果质量下降。


| 城市规模 | 运行次数 | 最小路径 | 标准路径 |
|----------|----------|----------|----------|
| 38       | 10       | 6655.58  | 6656     |
|          | 1        | 6756.69  |          |
|          | 1        | 6673.57  |          |
| 194      | 10       | 9803     |  9352    |
|          | 1        | 9898     |          |
| 734      | 10       | 85856    |   79114       |
|          | 1        | 86945    |          |
| 4663     |  1       |  1,448,278.54 |   1,290,319     |


### **3.22具体分析**

下面针对正式规模在$50 \sim 200$以内的数据进行详细分析

#### **(1)dj38.tsp**

该规模为50左右的城市数据，采用普通的$GA$算法，配置结果如下：
```python
    config = {
        'pop_size': 200,
        'generations': 20000,
        'crossover_rate': 0.9,
        'mutation_rate': 0.02,
        'elitism_count': 1,
        'use_two_opt': False,
        'gen_two_opt': 400,
        'two_opt_strategy': 'normal',
        'init_strategy': 'random',
        'selection_strategy': 'tournament',
        'crossover_strategy': 'mixed',
        'early_stop': 1000
    }
```
结果如下：\
$Best : 14010.64$ \
$Worst: 14010.64$ \
$Avg  : 14010.64$

![路径图](path_38cities_run1.png)

![收敛图](convergence_38cities_run1.png)

很显然在没有进行优化的配置下，程序离最优距离6656有一定的差距，在经过优化，采用局部搜索，精英保留策略以及初始化种群策略后，仅用少量的迭代次数就成功收敛，结果如下：

```python
    config = {
        'pop_size': 200,
        'generations': 2000,
        'crossover_rate': 0.9,
        'mutation_rate': 0.02,
        'elitism_count': 1,
        'use_two_opt': True,
        'gen_two_opt': 400,
        'two_opt_strategy': 'normal',
        'init_strategy': 'mixed',
        'selection_strategy': 'tournament',
        'crossover_strategy': 'mixed',
        'early_stop': 300
    }
```

$Best : 6659.43$ \
$Worst: 6659.43$ \
$Avg  : 6659.43$

![路径图](path_38cities_run1_2.png)

![收敛图](convergence_38cities_run1_2.png)


在小规模的城市中，初始种群的`mixed`策略混合了最邻近与邻近最远的两个优秀种群，其的解已经接近最优，导致最终过早收敛得到一个标准解。同时为了减少不必要的运算量，程序设置了提前终止，当累计`early_stop`代没有改进后程序终止。
故最终可以认为该$GA$算法对于小规模的城市数据有较好的解决.

#### **(2)qa194.tsp**

对于$200$规模左右的数据比如`qa194.tsp`，同样采用类似的做法，选择局部搜索以及`mixed`的种群策略，下面给出结果：

```python
    config = {
        'pop_size': 200,
        'generations': 2000,
        'crossover_rate': 0.9,
        'mutation_rate': 0.02,
        'elitism_count': 1,
        'use_two_opt': True,
        'gen_two_opt': 500,
        'two_opt_strategy': 'normal',
        'init_strategy': 'mixed',
        'selection_strategy': 'tournament',
        'crossover_strategy': 'mixed',
        'early_stop': 500
    }
```


$Best : 10028.73$ \
$Worst: 10028.73$ \
$Avg  : 10028.73$ 

![路径图](path_194cities_run1_2.png)

![收敛图](convergence_194cities_run1_2.png)

就结果而言，十分接近$9352$标准解，但同样的问题由于初始化种群中的优秀个体基因过于优秀，导致算法没有体现出进化的过程，调整参数`init_strategy=random`可以体现出进化的过程，但为了保证结果的正确性，需要加入局部搜索优化结果。

下面给出一个采用`init_strategy=random`的过程：

$Best : 10117.02$ \
$Worst: 10117.02$ \
$Avg  : 10117.02$ 



![收敛图](convergence_194cities_run1_3.png)
![路径图](path_194cities_run1_3.png)

可以明显看到，采用了局部优化策略后快速收敛，几乎在$100$代内达到最优解，对于平均解，由于加入了动态的变异率调整，导致在多样性发生衰减时变异率提升以应对早熟收敛，故平均值可能会出现中间上升的情况。

#### **(4)ca4663.tsp**

对于大规模的数据，普通的局部搜索由于算法时间复杂度为$O(n^3)$，需要消耗大量时间，故采用前文的`limited`模式的算法，只采取某个路径中部分节点翻转测试。同时初始化种群可以采用`mixed`形式的以减少收敛的时间，快速收敛得到一个较优的解。
需要注意的是，当数据规模增大时，进行计算需要的时间也在增大，$4000$的数据可能需要将近$10$个小时的运行时间。
```python
    config = {
        'pop_size': 300,
        'generations': 5000,
        'crossover_rate': 0.9,
        'mutation_rate': 0.02,
        'elitism_count': 1,
        'use_two_opt': True,
        'gen_two_opt': 1000,
        'two_opt_strategy': 'limited',
        'init_strategy': 'mixed',
        'selection_strategy': 'tournament',
        'crossover_strategy': 'mixed',
        'early_stop': 500
    }
```

运算结果如下：
$Best : 1448278.54$ \
$Worst: 1448278.54$ \
$Avg  : 1448278.54$ 

![收敛图](convergence_4663cities_run1_3.png)
![路径图](path_4663cities_run1_3.png)

由于代码在后期不断提高了变异率导致看到的评论路径增大，同时由于采用了优秀的初始种群，导致最佳路径很快达到最优从而有如上图的效果。对比标准数据$1,290,319$，差距在$10 \%$左右，在可以接受的范围内。

### **3.3性能分析**

本实验在普通的$GA$算法中加入了局部搜索，动态调整变异率，不同的交叉方式，以及设置不同的初始化种群，从而达到快速收敛并且具备一定的抗早熟收敛能力。在引入局部搜索的情况下，计算需要的时间大幅度增加，但是收敛速度快速提升。下面以`dj38.tsp`给出一个样例，为了达到较好的效果，经过多次运行总结得大致需要$20000$次的迭代。而采用优化的方案收敛速度很快(大致$2000$代就可以)，同时效果很好，但是面对大规模的数据时局部搜索的时间成本上升过快。

- 结果如下：\
    $Best : 7931.64$ \
    $Worst: 7931.64$ \
    $Avg  : 7931.64$ \
    运行的时间: $331.4206464290619$

- 同样的使用优化后的方案需要的时间以及结果如下：\
    $Best : 7233.13$ \
    $Worst: 7233.13$ \
    $Avg  : 7233.13$ \
    运行的时间: $15.041143894195557$

可以看到对于规模不大的数据，优化的参数设置可以减少大量的时间